In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# HiGAN training/generation

technical stuff

In [ ]:
!cp -r /content/drive/MyDrive/HWT/HiGAN /content/HiGAN #if HiGAN
%cd /content/HiGAN/HiGAN-main
!pip install -r requirements.txt
!pip install --force-reinstall torch==1.11.0
!pip install --force-reinstall protobuf==3.9.2

In [ ]:
!cp -r /content/drive/MyDrive/HWT/HiGAN+ /content/HiGAN+ #if HiGAN+
%cd /content/HiGAN+
!pip install munch distance
!pip install --force-reinstall torch==1.11.0
!pip install --force-reinstall protobuf==3.9.2

training

In [ ]:
!python train.py --config ./configs/gan_iam.yml

generating dataset reproducing words from cyrillic or notebooks dataset - which one is specified in configuraton file

In [ ]:
!python eval_demo.py --config ./configs/gan_iam.yml --ckpt ./ckpt/last1.pth --mode repro #if HiGAN

In [ ]:
!python eval_demo.py --config ./configs/gan_iam.yml --mode repro #if HiGAN+

#Preprocessing cyrillic handwriting dataset

In [ ]:
!cp /content/drive/MyDrive/HWT/archive.zip archive.zip
!unzip archive.zip

checking which symbols are present

In [ ]:
import pandas as pd
test=pd.read_table('test.tsv', header=None)
train=pd.read_table('train.tsv', header=None)
chars=set()
for i in train[1]:
  for j in str(i):
      chars.add(j)
for i in test[1]:
  for j in str(i):
      chars.add(j)
''.join(sorted(list(chars))), len(chars)

(' !"%\'()+,-./0123456789:;=?R[]abcehinoprstuxy«»АБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяё№',
 109)

processing cyrillic handwriting dataset to fit the input format for model training

In [ ]:
# stuff in this cell is taken from github.com/MuellerDominik/HiGANplus and slightly modified to fit the format of cyrillic dataset
import re
import itertools
import h5py
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
import os
from os import path
from PIL import Image

def gen_h5file(all_imgs, #saving in hdf format
               all_texts,
               all_wids,
               save_name):

    img_seek_idxs, img_lens = [], []
    cur_seek_idx = 0
    for img in all_imgs:
        img_seek_idxs.append(cur_seek_idx)
        img_lens.append(img.shape[-1])
        cur_seek_idx += img.shape[-1]

    lb_seek_idxs, lb_lens = [], []
    cur_seek_idx = 0
    for lb in all_texts:
        lb_seek_idxs.append(cur_seek_idx)
        lb_lens.append(len(lb))
        cur_seek_idx += len(lb)

    save_imgs = np.concatenate(all_imgs, axis=-1)
    save_texts = list(itertools.chain(*all_texts))
    save_lbs = [ord(ch) for ch in save_texts]
    save_path = os.path.join(save_name + '.hdf5')
    h5f = h5py.File(save_path, 'w')
    h5f.create_dataset('imgs',
                       data=save_imgs,
                       compression='gzip',
                       compression_opts=4,
                       dtype=np.uint8)
    h5f.create_dataset('lbs',
                       data=save_lbs,
                       dtype=np.int32)
    h5f.create_dataset('img_seek_idxs',
                       data=img_seek_idxs,
                       dtype=np.int64)
    h5f.create_dataset('img_lens',
                       data=img_lens,
                       dtype=np.int16)
    h5f.create_dataset('lb_seek_idxs',
                       data=lb_seek_idxs,
                       dtype=np.int64)
    h5f.create_dataset('lb_lens',
                       data=lb_lens,
                       dtype=np.int16)
    h5f.create_dataset('wids',
                       data=all_wids,
                       dtype=np.int16)
    h5f.close()
    print('save->', save_path)




# Choose whether the train or test set should be generated (uncomment needed)
is_test = True
# is_test = False
# data=train
data=test

# Choosing Input Directory
if is_test == True:
    input_dir = './test/'
else:
    input_dir = './train/'

all_wids = np.empty((0,))
all_texts = []
all_imgs = []
words_list = []

writers_dict={}
imsize=32 #set 32 for higan and 64 for higan+
save=True #if we want to replace original data in directory too, not only save in hdf5
for idx, line in data.iterrows():
        file_name=line[0]
        text=line[1]
        if not idx%200:
          print(idx, file_name)
        image = cv2.imread(os.path.join(input_dir, file_name))

        # Image Information:
        string_parts = str(text)
        word_length = len(string_parts)
        # Saving the words to generate a txt file
        word = string_parts
        words_list.append(word)

        # Numpy array with the writer ID's:
        author=re.sub(r'[^a-zA-Z]', '', file_name[:-4]) #author identifier is in file name, e.g. aa or yob
        if author in writers_dict:
          writer_ID=writers_dict[author]
        elif not writers_dict:
          writer_ID=0
          writers_dict[author]=0
        else:
          writer_ID=max(list(writers_dict.values()))+1
          writers_dict[author]=writer_ID
          print(writers_dict)
        writer_ID = int(writer_ID)
        all_wids = np.append(all_wids, writer_ID)

        # Numpy array with label text:
        all_texts.append(string_parts)

        # Convert the image to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # Denoiseing Image
        #  Apply erosion to the binary image
        denoised = cv2.erode(gray, (5, 5), iterations=1)
        # Threshold the image to binary
        _, binary = cv2.threshold(denoised, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        # Find the contours of the handwriting
        x, y, w, h = cv2.boundingRect(binary)
        cv2.rectangle(binary, (x, y), (x + w, y + h), (255, 0, 0), 2)
        # Crop the image to the bounding box
        handwriting = gray[y:y + h, x:x + w]
        # Resize the images while keeping the aspect ratio the same
        original_aspect_ratio = handwriting.shape[1] / handwriting.shape[0]
        resized_image = cv2.resize(handwriting, (int(imsize * original_aspect_ratio), imsize))
        # Inverting the image:
        _, resized_binary = cv2.threshold(resized_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        # Numpy matrix with all the pre-processed images:
        all_imgs.append(resized_binary)

        if save:
          _, resized_binary = cv2.threshold(resized_image, 0, 255, cv2.THRESH_OTSU)
          cv2.imwrite(os.path.join(input_dir, file_name), resized_binary)
print("end_of_dataset")

# Hdf5 File Generation:
if is_test == True:
    gen_h5file(all_imgs, all_texts, all_wids, '/content/drive/MyDrive/HWT/cyrillic/ru_hdf5_file_test')
else:
    gen_h5file(all_imgs, all_texts, all_wids, '/content/drive/MyDrive/HWT/cyrillic/ru_hdf5_file_train')


# Generating a txt file to test the model: c
if is_test == True:
    with open('/content/drive/MyDrive/HWT/cyrillic/ru_test_words.txt', 'w') as f:
        # Write each item in the list to the file on a new line
        for item in words_list:
            f.write(str(item) + '\n')
else:
    pass


0 test0.png
200 test1190.png
400 test1390.png
{'test': 0, 'testt': 1}
600 test168.png
800 test349.png
1000 test537.png
1200 test722.png
1400 test921.png
{'test': 0, 'testt': 1, 'testtt': 2}
end_of_dataset


# Notebooks

getting words from notebooks dataset - if pictures are also needed, uncomment commented parts

In [ ]:
import json
import numpy as np
import cv2
import pandas as pd
import os
%cd ./notebooks/
for group in ['train', 'test', 'val']:
  k=0
  with open('annotations_'+group+'.json') as f:
    dat=json.load(f)
  print(group, len(dat['annotations']))
  imgs={img['id']:img['file_name'] for img in dat['images']}
  crops={}
  prev=''
  for word in dat['annotations']:
    k+=1
    if not k%1000:
      print(k)
    if not 'attributes' in word: #if some polygon doesn't describe any word/symbolic sequence
      continue
    #forming new filename (old+number of word derived)
    filename=imgs[word['image_id']]
    if not filename in crops:
      crops[filename]=0
    else:
      crops[filename]=crops[filename]+1
    name=filename+'_'+str(crops[filename]) #+'.jpg'
    # if os.path.exists(group+'/'+name):
    #   continue
    # if not filename==prev: #to read img only once
    #   img = cv2.imread('./images/images/'+filename)
    # prev=filename
    #getting text
    text=word['attributes']['translation']
    # #cutting polygon with word from notebook page and saving in distinct file
    # mask = np.zeros(img.shape[0:2], dtype=np.uint8)
    # xs=word['segmentation'][0][::2]
    # ys=word['segmentation'][0][1::2]
    # points = np.array([[[int(xs[i]), int(ys[i])] for i in range(len(xs))]])
    # cv2.drawContours(mask, [points], -1, (255, 255, 255), -1, cv2.LINE_AA)
    # res = cv2.bitwise_and(img,img,mask = mask)
    # rect = cv2.boundingRect(points) # returns (x,y,w,h) of the rect
    # cropped = res[rect[1]: rect[1] + rect[3], rect[0]: rect[0] + rect[2]]
    # wbg = np.ones_like(img, np.uint8)*255
    # cv2.bitwise_not(wbg,wbg, mask=mask)
    # dst = wbg+res
    # dst=dst[rect[1]: rect[1] + rect[3], rect[0]: rect[0] + rect[2]]
    # cv2.imwrite(group+'/'+name, dst)

    #making index
    w=pd.DataFrame([[name, text]], columns=['file', 'text'])
    w.to_csv('/content/'+group+'v2'+'.csv', mode='a', header=not os.path.exists('/content/'+group+'v2'+'.csv'))

# Eval

installations and imports

In [ ]:
!pip install torchmetrics
!pip install torchmetrics[image]
!conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.8
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git


In [1]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
import pandas as pd
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms
import os
from torchmetrics.image.fid import FrechetInceptionDistance
import torch.nn as nn
import torchvision.transforms as T
import clip
from PIL import Image
import warnings
import numpy as np
warnings.filterwarnings('ignore')

getting data

In [ ]:
!cp /content/drive/MyDrive/HWT/archive.zip archive.zip
!unzip archive.zip
!cp /content/drive/MyDrive/HWT/cyrillic_repro.zip repro.zip
!unzip repro.zip

In [ ]:
!cp /content/drive/MyDrive/HWT/cyrillic_repro+.zip repro.zip
!unzip repro.zip

reading index

In [2]:
real_train=pd.read_table('train.tsv', header=None)
real_test=pd.read_table('test.tsv', header=None)
fake_test=pd.read_csv('test.csv')
fake_test=fake_test[fake_test['file'].str.endswith('0.png')][['file', 'text']]
fake_train=pd.read_csv('train.csv')
fake_train=fake_train[fake_train['file'].str.endswith('0.png')]

counting letters

In [ ]:
letters=[j for i in real_train[1].values for j in str(i)]
cnt=dict(Counter(letters))
for i in sorted(cnt, key=cnt.get):
  print(i, cnt[i], round(cnt[i]/len(letters)*100, 2))

In [3]:
class DS(Dataset):
    def __init__(self, labels, root_dir, subset=False, transform=None):
        self.labels = labels
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        img_name = self.labels.iloc[idx, 0]
        fullname = os.path.join(self.root_dir, img_name)
        image = Image.open(fullname).convert('RGB')
        labels = self.labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        return [image, labels]

## FID

conventional FID calculation

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
]) #how to preprocess data to fit inception input
real = DS(real_train, 'train', transform=preprocess)
fake= DS(fake_train, 'train', transform=preprocess)
bs=128
nw=2
real_dl = DataLoader(real, batch_size=bs, shuffle=True, num_workers=nw)
fake_dl = DataLoader(fake, batch_size=bs, shuffle=True, num_workers=nw)
fid = FrechetInceptionDistance(normalize=True).to(device)
k=0
#going through real and fake data to update means and covariance online
for x, y in real_dl:
  k+=1
  print(k)
  fid.update(x.to(device), real=True)
k=0
for x, y in fake_dl:
  k+=1
  print(k)
  fid.update(x.to(device), real=False)
fid.compute() #finally, calculating FID

adding test data up to train

In [ ]:
real = DS(real_test, 'test', transform=preprocess)
fake= DS(fake_test, 'test', transform=preprocess)
real_dl = DataLoader(real, batch_size=bs, num_workers=nw)
fake_dl = DataLoader(fake, batch_size=bs, num_workers=nw)
k=0
for x, y in real_dl:
  k+=1
  print(k)
  fid.update(x.to(device), real=True)
k=0
for x, y in fake_dl:
  k+=1
  print(k)
  fid.update(x.to(device), real=False)
fid.compute()

test without train

In [ ]:
fid = FrechetInceptionDistance(normalize=True).to(device)
k=0
for x, y in real_dl:
  k+=1
  print(k)
  fid.update(x.to(device), real=True)
k=0
for x, y in fake_dl:
  k+=1
  print(k)
  fid.update(x.to(device), real=False)
fid.compute()

## Clip

same things as for conventional FID but CLIP in wrapper instead of inception

In [ ]:
model, preprocess = clip.load("ViT-B/32", device=device)
class ClipWrapper(nn.Module):
  def __init__(self, clip, preprocess):
    super().__init__()
    self.clip=clip
    self.prefix=prefix
    self.preprocess=preprocess
    self.transform = T.ToPILImage()
  def forward(self, x):
    batch_features=[]
    with torch.no_grad():
      if x.shape[0]==1: #when initialising FID, it passes dummy input through model, this is for this dummy input to fit CLIP
        x=self.transform(x[0])
        image = self.preprocess(x).unsqueeze(0).to(device)
        image_features = model.encode_image(image)
        batch_features.append(image_features)
        return torch.cat(batch_features)
      else:
        return model.encode_image(x)
    
clip_w=ClipWrapper(model, preprocess)

fid = FrechetInceptionDistance(feature=clip_w, reset_real_features=True).to(device)

In [ ]:
real = DS(real_train, 'train', transform=preprocess)
fake= DS(fake_train, 'train', transform=preprocess)
bs=128
nw=2
real_dl = DataLoader(real, batch_size=bs, shuffle=True, num_workers=nw)
fake_dl = DataLoader(fake, batch_size=bs, shuffle=True, num_workers=nw)
k=0
for x, y in real_dl:
  k+=1
  print(k)
  fid.update(x.to(device), real=True)
k=0
for x, y in fake_dl:
  k+=1
  print(k)
  fid.update(x.to(device), real=False)
fid.compute()

In [ ]:
real = DS(real_test, 'test', transform=preprocess)
fake= DS(fake_test, 'test', transform=preprocess)
real_dl = DataLoader(real, batch_size=bs, shuffle=True, num_workers=nw)
fake_dl = DataLoader(fake, batch_size=bs, shuffle=True, num_workers=nw)
k=0
for x, y in real_dl:
  k+=1
  print(k)
  fid.update(x.to(device), real=True)
k=0
for x, y in fake_dl:
  k+=1
  print(k)
  fid.update(x.to(device), real=False)
fid.compute()

In [ ]:
fid = FrechetInceptionDistance(feature=clip_w, reset_real_features=True).to(device)
k=0
for x, y in real_dl:
  k+=1
  print(k)
  fid.update(x.to(device), real=True)
k=0
for x, y in fake_dl:
  k+=1
  print(k)
  fid.update(x.to(device), real=False)
fid.compute()